In [12]:
import numpy as np
import pandas as pd
import os,cv2
from keras.utils import image_dataset_from_directory

In [13]:
df="/kaggle/input/chinese-landscape-painting-dataset"

In [15]:
train_data=image_dataset_from_directory(
df,
batch_size=32,
image_size=(256,256),
seed=123,
shuffle=True,
label_mode=None
)

Found 2192 files.


# # GAN

In [31]:
from keras.layers import Dense,Conv2D,MaxPooling2D,BatchNormalization,Dropout,Input,Reshape,Conv2DTranspose,ReLU,Flatten
from keras.models import Model
from keras.regularizers import l2
from keras.initializers import HeNormal
from keras.callbacks import EarlyStopping

In [32]:
def build_gen():
    inputs=Input(shape=(100,))
    C=Dense(16*16*256)(inputs)
    C=Reshape((16,16,256))(C)
    C=Conv2DTranspose(256,(3,3),strides=(2,2),padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(C)
    C=ReLU()(C)
    C=Dropout(0.5)(C)
    C=BatchNormalization()(C)
    C=Conv2DTranspose(128,(3,3),strides=(2,2),padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(C)
    C=ReLU()(C)
    C=Dropout(0.5)(C)
    C=BatchNormalization()(C)
    C=Conv2DTranspose(64,(3,3),strides=(2,2),padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(C)
    C=ReLU()(C)
    C=Dropout(0.5)(C)
    C=BatchNormalization()(C)
    C=Conv2DTranspose(32,(3,3),strides=(2,2),padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(C)
    C=ReLU()(C)
    C=Dropout(0.5)(C)
    C=BatchNormalization()(C)
    outputs=Conv2D(3,(3,3),activation="tanh",padding='same')(C)
    model=Model(inputs,outputs)
    return model
gen=build_gen()
gen.summary()

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_9 (InputLayer)      │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 65536)          │     6,619,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_5 (Reshape)             │ (None, 16, 16, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_13             │ (None, 32, 32, 256)    │       590,080 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_20 (ReLU)                 │ (None, 32, 32, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_20 (Dropout)            │ (None, 32, 32, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_20          │ (None, 32, 32, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_14             │ (None, 64, 64, 128)    │       295,040 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_21 (ReLU)                 │ (None, 64, 64, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_21 (Dropout)            │ (None, 64, 64, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_21          │ (None, 64, 64, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_15             │ (None, 128, 128, 64)   │        73,792 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_22 (ReLU)                 │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_22 (Dropout)            │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_22          │ (None, 128, 128, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_16             │ (None, 256, 256, 32)   │        18,464 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_23 (ReLU)                 │ (None, 256, 256, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_23 (Dropout)            │ (None, 256, 256, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_23          │ (None, 256, 256, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 256, 256, 3)    │           867 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,599,299 (28.99 MB)

 Trainable params: 7,598,339 (28.99 MB)

 Non-trainable params: 960 (3.75 KB)

In [33]:
def build_dis():
    inputs=Input(shape=(256,256,3))
    C=Conv2D(32,(3,3),strides=(2,2),padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(inputs)
    C=ReLU()(C)
    C=Dropout(0.5)(C)
    C=BatchNormalization()(C)
    C=Conv2D(64,(3,3),strides=(2,2),padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(C)
    C=ReLU()(C)
    C=Dropout(0.5)(C)
    C=BatchNormalization()(C)
    C=Conv2D(128,(3,3),strides=(2,2),padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(C)
    C=ReLU()(C)
    C=Dropout(0.5)(C)
    C=BatchNormalization()(C)
    C=Conv2D(32,(3,3),strides=(2,2),padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(C)
    C=ReLU()(C)
    C=Dropout(0.5)(C)
    C=BatchNormalization()(C)
    C=Flatten()(C)
    C=Dense(32,activation="relu")(C)
    outputs=Dense(1,activation="sigmoid")(C)
    model=Model(inputs,outputs)
    return model
dis=build_dis()
dis.summary()

Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_10 (InputLayer)     │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 128, 128, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_24 (ReLU)                 │ (None, 128, 128, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_24 (Dropout)            │ (None, 128, 128, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_24          │ (None, 128, 128, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 64, 64, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_25 (ReLU)                 │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_25 (Dropout)            │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_25          │ (None, 64, 64, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 32, 32, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_26 (ReLU)                 │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_26 (Dropout)            │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_26          │ (None, 32, 32, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_16 (Conv2D)              │ (None, 16, 16, 32)     │        36,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_27 (ReLU)                 │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_27 (Dropout)            │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_27          │ (None, 16, 16, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 32)             │       262,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 393,377 (1.50 MB)

 Trainable params: 392,865 (1.50 MB)

 Non-trainable params: 512 (2.00 KB)

In [35]:
def build_gan(generator,discriminator):
    discriminator.trainable=False
    inputs=Input(shape=(100,))
    fake_img=generator(inputs)
    outputs=discriminator(fake_img)
    model=Model(inputs,outputs)
    return model
gan=build_gan(gen,dis)
gan.summary()

Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_11 (InputLayer)     │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ functional_7 (Functional)       │ (None, 256, 256, 3)    │     7,599,299 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ functional_9 (Functional)       │ (None, 1)              │       393,377 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,992,676 (30.49 MB)

 Trainable params: 7,598,339 (28.99 MB)

 Non-trainable params: 394,337 (1.50 MB)

In [ ]:
def train_gan(gan,gen,dis,epochs,batch_size,latent_dim,dataset):
    dis.trainable=True
    dis.compile(optimizer='adam',loss="binary_crossentropy",metrics=['accuracy'])
    gan.compile(optimizer='adam',loss="binary_crossentropy",metrics=["accuracy"])
    for epochs in range(epochs):
        for real_img in dataset:
            real_label=np.ones((batch_size,1))
            fake_label=np.zeros((batch_size,1))
            noise=np.random.normal(0,1,(batch_size,latent_dim))
            fake_img=gen(noise)
            dis.trainable=True
            dis.train_on_batch(real_img,real_label)
            dis.train_on_batch(fake_img,fake_label)
            dis.trainable=False
            g_g=gan.train_on_batch(noise,real_label)
    return g_g
train_gan=train_gan(gan,gen,dis,3,32,100,train_data)